This notebook will take an in folder of raw unzipped Landsat geotiff images from the USGS and merge them into 
a single multiband image. 

Gonna add a .yml file for imports, but if you're using Conda these shouldn't be much of a 
problem. conda install numpy, then matplotlib, then gdal and you're set.

In [1]:
import numpy as np
from osgeo import gdal, ogr, osr
import matplotlib.pyplot as plt
import os

I suggest putting your raw landsat unzipped data in a folder titled 'data' one directory above
the 'scripts' directory, as the os.walk function in the next cell will look through a certain folder
for landsat images ending with the specified band combos. See this link for more info:
https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites?qt-news_science_products=0
And specify an out_tif_name for the out file name


In [2]:
in_tif_dir = r'../../data'
out_tif_name = 'Austin_Landsat' + '.tif'
tif_driver = gdal.GetDriverByName('GTiff')
out_tif_path = os.path.join(in_tif_dir, out_tif_name)

Obviously I can streamline this portion, I love my os.walks instead of glob for some reason :D
I read in Red, then Blue, then Green for a typical RGB band combination
Then open via GDAL and read each as an array

In [3]:
tif_dir_list = []
for dirs, subdirs, files in os.walk(in_tif_dir):
    for file in files:
        fname = file.split('.')[0]
        if fname.endswith('B4'):
            #Red
            path = os.path.join(dirs, file)
            ds = gdal.Open(path)
            arr = ds.GetRasterBand(1).ReadAsArray()       
            tif_dir_list.append(arr)
        elif fname.endswith('B3'):
            #Green
            path = os.path.join(dirs, file)
            ds = gdal.Open(path)
            arr = ds.GetRasterBand(1).ReadAsArray()  
            tif_dir_list.append(arr)
        elif fname.endswith('B2'):
            #Blue
            path = os.path.join(dirs, file)
            ds = gdal.Open(path)
            arr = ds.GetRasterBand(1).ReadAsArray()
            tif_dir_list.append(arr)
        elif fname.endswith('B5'):
            #Near IR
            path = os.path.join(dirs, file)
            ds = gdal.Open(path)
            arr = ds.GetRasterBand(1).ReadAsArray()
            tif_dir_list.append(arr)

Sets up the GDAL magic
First opens the geotiff, then stores the geotransform and projection info

In [4]:
in_ds = gdal.Open(path)
in_gt = in_ds.GetGeoTransform()
prj = in_ds.GetProjection()
srs = osr.SpatialReference(wkt=prj)

Creates an out tif with the info you provided in cell 2 as well as 
predetermined variables projection and geotransform

In [5]:
out_tif = tif_driver.Create(out_tif_path, 
                            tif_dir_list[0].shape[1],
                            tif_dir_list[0].shape[0],
                            len(tif_dir_list),
                            gdal.GDT_Float64)
out_tif.SetProjection(srs.ExportToWkt())
out_tif.SetGeoTransform(in_gt);

GDAL suxx and starts indexing at 1, instead of good ole 0
so we loop through each stored array in the list and write as a separate color band

In [6]:
i = 1
for arr in tif_dir_list:    
    out_band = out_tif.GetRasterBand(i)
    out_band.WriteArray(arr)
    i+=1
out_tif.FlushCache()
del out_tif